# overview
This jupyternote shows how to have basic setting for performing the Quantum State Tomography (QST) by using one of the following optimization approach:   

1. Riemannian Gradient Descent (RGD)
2. Momentum-Inspired Factored Gradient Descent (MiFGD)



In [10]:
import Get_param

from Utility import State_Naming
from Utility import Params_define

from Input_Utility import Gen_Projectors
from Input_measurements import Get_measurement_by_labels

#from main_QST import Default_Setting_Run_Tomography
from main_QST import Default_OptMethod_params, Tomography_over_measurement
from main_QST import Tune_Kappa_Tomography


In [11]:
#Nk = 6                      #  number of qubits
#m = int(0.3* (4**Nk))       #  number of sampled Pauli matrices
#mea = 8600                  #  mea =  number of shots

Nk = 3                      #  number of qubits
m = 20                      #  number of sampled Pauli matrices
mea = 2400                  #  mea =  number of shots

State_version = { 'StateName': 'GHZ',        #  = 'GHZ', 'Had', 'rand', 'KapRnd'
                      'Nr': 1,                    #  rank: 'GHZ', 'Had' --> only allow Nr =1 
                      'Generate New rand': 1,     #  (only for 'rand', 'KapRnd') 1: generate new data |  0: load existing data
                      'rand matrix version': 1,   #  (only for 'rand', 'KapRnd') version for random matrices  
                      'List_alpha': [5, 7, 10]    #  only for 'KapRnd' to tune Kappa (not needed for other states)
    }

Version_Def = { 
        'Gen New Proj sampling': 1,     #  -1: specify fixed list, 1: generate sampling, 0: load existing sampling
        'Proj version': 1,              #  counting projector version number
        'Gen New Measure': 1,           #  1: generate new shot/calculated measure, 0: load existing 
        'measure version': 2,           #  counting measure version number
    }


In [12]:
Pj_method = 1               #   the method to save | load  projectors
mea_method = 1              #   the method to save | load  measurement_dict (count_dict) 

measure_method = 3          #  = 1: direct label_list,  = 3: parallel cpu

DirStore = './DataTest'

## Define params_setup
- store all parameters in the dictionary params_setup

In [13]:
params_setup = Params_define(State_version, Nk, m, mea,\
                Pj_method, mea_method, measure_method, DirStore) 

params_setup = State_Naming(params_setup, \
                State_version, Version_Def)

In [14]:
params_setup

{'n': 3,
 'Name': 'GHZ-3',
 'StateName': 'GHZ',
 'num_labels': 20,
 'num_shots': 2400,
 'Nr': 1,
 'Noise': -1,
 'Dir0': './DataTest/GHZ-3',
 'Pj_method': 1,
 'mea_method': 1,
 'measure_method': 3,
 'Gen New Proj sampling': 1,
 'Proj version': 1,
 'Gen New Measure': 1,
 'measure version': 2,
 'Data_In': [],
 'DirRho': './DataTest/GHZ-3/GHZ-3_m20_s1',
 'Dir2p': './DataTest/GHZ-3',
 'projector_store_path': './DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1_Proj',
 'Dir2m': './DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1',
 'measurement_store_path': './DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1_shot2400_v2_Measure'}

## generate the projectors = Pauli matrices 

In [15]:
label_list, T_rec, Ncpu_Pj, saveP_bulk  = Gen_Projectors(params_setup)  # creating projectors

 -------------     Generate new labels    ------------
      Partion_Pj = 0  -->  No partion in Pj

  ***  Pj_file    = ./DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1_Proj/Pj_list.pickle  is saved (i.e. dump)
  ***  label_file = ./DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1_Proj/labels.pickle   is saved

     pool.map  COMPLETED by #CPU = 16 

     [projectors] num_cpus = 16,  saved bulk Pj = 1 


 *****        num_cpus = 16, saved # bulk Pj = 1        ****

 *****   The running time for projector = 1.4340870380401611  **** 

    label_list[:10]   = ['ZZX', 'XXZ', 'IYX', 'YZY', 'XYY', 'IXX', 'XXI', 'YYY', 'ZYZ', 'IYI']

    num of projectors = len(label_list) = 20
    num of saved bulk Proj              = 1


  <<<<<<<<                Do_projector time    =  1.4351449012756348           >>>>>>

  ######       saveP_bulk = 1  for Ncpu_Pj = 16    ########



## Generate the state and do measurements

In [16]:
target_density_matrix, input_S, T_rec, Ncpu_meas = \
        Get_measurement_by_labels(params_setup, label_list, T_rec)

  ********   starting to DO measurement (qiskit or qutip)  ************** 

 num_cpus = 16,  labels size =20
       saving 0-th label_part DONE
       saving 1-th label_part DONE
       saving 2-th label_part DONE
       saving 3-th label_part DONE
       saving 4-th label_part DONE
       saving 5-th label_part DONE
       saving 6-th label_part DONE
       saving 7-th label_part DONE
       saving 8-th label_part DONE
       saving 9-th label_part DONE
       saving 10-th label_part DONE
       saving 11-th label_part DONE
       saving 12-th label_part DONE
       saving 13-th label_part DONE
       saving 14-th label_part DONE
       saving 15-th label_part DONE

 *****************   saving label_part files DONE  *********** 


  ml_lab_files = ['ml_labels_10', 'ml_labels_11', 'ml_labels_4', 'ml_labels_3', 'ml_labels_2', 'ml_labels_5', 'ml_labels_13', 'ml_labels_14', 'ml_labels_15', 'ml_labels_12', 'ml_labels_9', 'ml_labels_0', 'ml_labels_7', 'ml_labels_6', 'ml_labels_1', 'ml_label

## Do the tomography
- pm_MiFGD: parameters for running MiFGD
- pm_RGD: parameters for running RGD

In [17]:
pm_MiFGD, pm_RGD = Default_OptMethod_params()

Ttot, Wk_dict, params_setup, T_rec = \
        Tomography_over_measurement(params_setup, target_density_matrix, \
                             input_S, pm_RGD, pm_MiFGD, T_rec)

 ********************************************************* 

   there exists pure state   


 ++++++++++++++++++++++++++++++++++++++++++++++++++++++ 
 ++           start calculating  RGD                 ++ 
 ++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

  self.Noise = shot
  Pj_method  = 1
  mea_method = 1
  --> loading measurement_list DONE from  ml_file = ./DataTest/GHZ-3/GHZ-3_m20_s1/GHZ-3_m20_s1_shot2400_v2_Measure/measureL_ALL.pickle

Loading projectors
   20 projectors loaded
   Projectors ready to compute with


  ******  projectors  -->  time = 0.00027179718017578125   ****** 

   ===>  to GET measurment_list  

Loading measurements
   Measurements ready to compute with

   **** Loading measurements   -->   time = 5.9604644775390625e-06  **** 

  self.coef = 0.6324555320336759
  InitX     = 1
 *********   using randomized-SVD  to construct  X0 = uk @ sDiag @ vkh  **************************
  ***   0-th (A A*) done:   Time --> ATQ: 9.298324584960938e-05,  AG: 9.393692

### Run various Kappa cases for StateName == 'KapRnd'
- List_alpha: parameters of different Kappa for StateName == 'KapRnd' only

In [18]:
if params_setup['StateName'] == 'KapRnd':
    T_Rho, Wk_Rho = Tune_Kappa_Tomography(params_setup, target_density_matrix, \
            input_S, T_rec, Ncpu_Pj, pm_MiFGD, pm_RGD)